In [1]:
from settings import RABBIT_ACCOUNT, RABBIT_PASSWORD, RABBIT_HOST, RABBIT_PORT, EVENT_QUEUE

In [2]:
import pandas as pd

In [3]:
from datetime import datetime

In [4]:
!ls

AEStatistic.ipynb	event_test.csv	      positivepage.csv
Event_auto_throw.ipynb	eventsourcepage.csv   publisher.py
__init__.py		fileDownloader.ipynb  settings.py
__pycache__		needCrawl.csv	      test_queue.ipynb
database.py		pika_queue.py	      worker.py
download		pika_queue.pyc


In [5]:
import pika
import json
class Queue():
    def __init__(self, host, port, username, password):
        self.connection = pika.BlockingConnection(pika.ConnectionParameters(host,port=port,credentials=pika.PlainCredentials(username=username,password=password), heartbeat=0))
        self.channel = self.connection.channel()
        self.channel.basic_qos(prefetch_count=1)
    def DeclareQueue(self, queuename):
        self.channel.queue_declare(queue = queuename)
    def AddToQueue(self, queuename, data):
        self.channel.basic_publish(exchange = '', routing_key = queuename, body = json.dumps(data), properties = pika.BasicProperties(delivery_mode=2))
    def GetFromQueue(self, queuename, callback, auto_ack):
        self.channel.basic_consume(queue = queuename, on_message_callback = callback,auto_ack= auto_ack)
        self.channel.start_consuming()
    def Close(self):
        self.connection.close()

In [6]:
event_queue = Queue(RABBIT_HOST, RABBIT_PORT, RABBIT_ACCOUNT, RABBIT_PASSWORD)

In [7]:
EVENT_QUEUE

'EventSourceURL'

In [27]:
import csv

In [35]:
event_df = pd.read_csv('event_test.csv')

In [36]:
event_df

,File Name,Page URL
0,1,https://ysmb.wda.gov.tw/
1,2,https://ysmb.wda.gov.tw/about.php
2,3,https://ysmb.wda.gov.tw/news.php
3,4,https://ysmb.wda.gov.tw/news.php?page=4&id=1
4,5,https://ysmb.wda.gov.tw/activity_all.php
...,...,...
95,96,https://tydep-eew.com.tw/news.php
96,97,https://dac.tw/currentcoming/
97,98,https://www.chcsec.gov.tw/informationlist?uid=...
98,99,https://event.culture.tw/CHCSEC/portal/Index/I...


In [29]:
# event_df = pd.read_csv('eventsourcepage.csv')
event_df = pd.read_csv('event_test.csv')
# event_df = pd.read_csv('positivepage.csv')
positivepage = []
with open('positivepage.csv', newline='') as f:
    reader = csv.reader(f) 
    tmp = list(reader)
    #print(p)
    positivepage += [','.join(p[0:-1]) for p in tmp if ':' in ','.join(p[0:-1])]

In [8]:
import validators

In [13]:
test_pages = [
    'https://www.city.hachioji.tokyo.jp/kurashi/shimin/004/003/001/p000099.html',
    'https://aia.kcg.gov.tw/siscity-1126-sistercitytw',
    'https://khh.travel/ja/event/calendar/2020',
    'https://www.hkc.or.jp/saijiki/index.html',
    'https://hachi-navi.com/event_detail/396',
    'https://www.pref.chiba.lg.jp/kokusai/kokusaikouryuu/exchange/taoyuan/index.html',
    'https://www.tycg.gov.tw/eng/home.jsp?id=17&parentpath=0,1,16',
    'https://maruchiba.jp/sys/search/jp/list?sid=10&sort=2&limit=20',
    'https://travel.tycg.gov.tw/ja/travel/season'
]

In [14]:
print("Adding urls to Event_queue...")
for url in test_pages:
    if validators.url(url):
#         print(url)
        event_queue.AddToQueue(EVENT_QUEUE, url)
#         break
#         pass
    else:
        print(url)
print("Finish adding url to Event_queue...")

Adding urls to Event_queue...
Finish adding url to Event_queue...


In [19]:
print("Adding urls to Event_queue...")
for url in event_df[col].values.tolist():
    if validators.url(url):
#         print(url)
        event_queue.AddToQueue(EVENT_QUEUE, url)
#         break
#         pass
    else:
        print(url)
print("Finish adding url to Event_queue...")

Adding urls to Event_queue...
Finish adding url to Event_queue...


In [15]:
from datetime import datetime, tzinfo, timedelta
class simple_utc(tzinfo):
    def tzname(self,**kwargs):
        return "UTC"
    def utcoffset(self, dt):
        return timedelta(0)
datetime.utcnow().replace(tzinfo=simple_utc()).isoformat()

'2021-07-09T14:29:48.903329+00:00'